In [70]:
import torch
import torchvision
import PIL
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder
import torchvision.transforms.functional as fn
from torch import nn
import torch.nn.functional as F
import wandb
import torch.optim as optim

In [11]:
csv_file = pd.read_csv('../../files/train.csv')
csv_file['updated_paths'] = csv_file['image'].apply(lambda x: '../../files/train_images/' + x)

In [35]:
def split_datasets(csv_file, test_size):
    train, test = train_test_split(csv_file, test_size=test_size)
    train, val = train_test_split(train, test_size=test_size)
    return train, val, test

In [36]:
def encoding_data(csv_data):
    labels = csv_data['species'].values.tolist()
    encoder = OneHotEncoder()
    encoder = encoder.fit(np.array(labels).reshape(-1, 1))
    return encoder

In [37]:
encoder = encoding_data(csv_file)

In [55]:
class DolphinDataset(Dataset):
    def __init__(self, csv_file, encoder):
        self.csv_file = csv_file
        self.images = csv_file['updated_paths'].values.tolist()
        self.labels = csv_file['species'].values.tolist()
        self.encoder = encoder
        self.encoded_labels = self.encoder.transform(np.array(self.labels).reshape(-1, 1)).toarray()

    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        img = torchvision.io.read_file(self.images[idx])
        img = torchvision.io.decode_jpeg(img)
        img = fn.resize(img, [512, 512])
        img = img / 255.0
        return img, self.encoded_labels[idx]

In [ ]:
train, val, test = split_datasets(csv_file)

In [67]:
incep_model

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [68]:
incep_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained = False)
incep_model.fc = nn.Linear(2048, 30)

Using cache found in /home/ubuntu/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/lib/python3/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/lib/python3/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/lib/python3/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [ ]:
def train_model(model, train_dataset, val_dataset, epochs, load_weights=None):
    wandb.init(project='ship-segmentation-pytorch-wb',
               config = {
                   'arch' : 'Unet- Incepv4',
                   'epochs' : epochs,
                   'where_weights_stored' : weights_path,
                   'weights_loaded': False if load_weights == None else load_weights,
                   'optimizer' : 'SGD',
                   'lr_schdular' : False,
                   'Data augmentation' : True,
                   'batch_size': train_batch_size,
                   'num_workers' : num_workers
               }
               )
    data_pointers = {
        'train' : train_dataset,
        'val' : val_dataset,
    }
    if load_weights != None:
        model.load_state_dict(torch.load(load_weights)['model_state_dict'])
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    if next(model.parameters().cuda) == False:
        if multiple_gpus == True:
            model = nn.DataParallel(model)
        model = model.to(device)
    for epoch in range(epochs):
        train_loss, train_precision, train_recall = 0.0, 0.0, 0.0
        val_loss, val_precision, val_recall = 0.0, 0.0, 0.0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss, running_precision, running_recall = 0.0, 0.0, 0.0
            